In [5]:
import pandas as pd
import numpy as np

1\. Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named "data_int.txt". Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named "data_float.txt". Use the `cat` command to print the content of the file.
+ load the txt file of the previous point and convert it to a csv file by hand.

In [1]:
ls = np.random.randint(100, size = 12)
f = open('data_int.txt', 'w')
for num in ls:
    f.write('%s ' % num)
f.close()
# !type command is equivalent of Unix !cat command on Windows
!type data_int.txt
print('--')
m = np.random.rand(5,5) * 100
f = open('data_float.txt','w')
for row in m:
    for num in row:
        f.write('%s ' % num)
    f.write('\n')
f.close()
!type data_float.txt
print('--')
data = np.loadtxt('data_float.txt')
f = open('data.csv','w')
for row in data:
    for num in row[:-1]:
        f.write('%s,' % num)
    f.write('%s\n' % row[-1])
f.close()
!type data.csv

NameError: name 'np' is not defined

2\. Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [4]:
# !wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json -P ./data/

import urllib.request as ur
u = ur.urlopen('https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json?dl=1')
data = u.read()
u.close()
with open('data/user_data.json','wb') as f:
    f.write(data)

df = pd.read_json('data/user_data.json')
df[df['CreditCardType'] == 'American Express'].to_csv('user_data.csv')

# df2 = pd.read_csv('user_data.csv')
# df2

FileNotFoundError: [Errno 2] No such file or directory: 'data/user_data.json'

3\. Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [3]:
df = pd.read_csv('data/mushrooms_categorized.csv')
gb = df.groupby('class').mean()
gb.to_json('out.json')
# !type out.json

NameError: name 'pd' is not defined

4\. Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

5\. **Optional**: load the remote file:

- https://www.dropbox.com/s/aamg1apjhclecka/regression_generated.csv

with Pandas and create a scatter plot with all possible combinations of the following features:
    
  + features_1
  + features_2
  + features_3